<a href="https://colab.research.google.com/github/Codeptor/DNA-Sequencing/blob/main/minor_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DNA Sequence Error Detection using Pysam: A Comprehensive Guide


## 1. Dataset Selection and Preparation
For this guide, we'll use a smaller dataset from the Sequence Read Archive (SRA) that's more manageable for research projects with limited computational resources.

### 1.1 Download a Small Dataset
We'll use a dataset from the E. coli genome, which is much smaller in size.



1. Install the SRA Toolkit:


In [12]:
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
!tar -xzf sratoolkit.current-ubuntu64.tar.gz
!export PATH=$PATH:$PWD/sratoolkit.3.0.0-ubuntu64/bin
!sudo apt-get install sra-toolkit

--2024-08-29 09:45:35--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.10, 2607:f220:41e:250::7, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93532905 (89M) [application/x-gzip]
Saving to: ‘sratoolkit.current-ubuntu64.tar.gz.2’

sratoolkit.current- 100%[===================>]  89.20M  32.0MB/s    in 2.8s    

2024-08-29 09:45:38 (32.0 MB/s) - ‘sratoolkit.current-ubuntu64.tar.gz.2’ saved [93532905/93532905]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  blends-common libkdf5-2 libncbi-vdb2 libncbi-wvdb2 med-config menu
Suggested packages:
  blends-doc menu-l10n gksu | kde-runtime | ktsuss
The following NEW packages will be installed:
  

2. Download a small E. coli dataset:


In [13]:
!prefetch SRR1770413
!fastq-dump --split-files SRR1770413


2024-08-29T09:45:59 prefetch.2.11.3: Current preference is set to retrieve SRA Normalized Format files with full base quality scores.
2024-08-29T09:46:00 prefetch.2.11.3: 1) Downloading 'SRR1770413'...
2024-08-29T09:46:00 prefetch.2.11.3: SRA Normalized Format file is being retrieved, if this is different from your preference, it may be due to current file availability.
2024-08-29T09:46:00 prefetch.2.11.3:  Downloading via HTTPS...
2024-08-29T09:46:06 prefetch.2.11.3:  HTTPS download succeed
2024-08-29T09:46:07 prefetch.2.11.3:  'SRR1770413' is valid
2024-08-29T09:46:07 prefetch.2.11.3: 1) 'SRR1770413' was downloaded successfully
2024-08-29T09:46:07 prefetch.2.11.3: 'SRR1770413' has 0 unresolved dependencies
Read 643253 spots for SRR1770413
Written 643253 spots for SRR1770413


This will give you two FASTQ files: `SRR1770413_1.fastq` and `SRR1770413_2.fastq`.



### 1.2 Download the Reference Genome
Download the E. coli reference genome:


In [14]:
!wget ftp://ftp.ensemblgenomes.org/pub/bacteria/release-45/fasta/bacteria_0_collection/escherichia_coli_str_k_12_substr_mg1655/dna/Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa.gz
!gunzip Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa.gz

--2024-08-29 09:46:29--  ftp://ftp.ensemblgenomes.org/pub/bacteria/release-45/fasta/bacteria_0_collection/escherichia_coli_str_k_12_substr_mg1655/dna/Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa.gz
           => ‘Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa.gz’
Resolving ftp.ensemblgenomes.org (ftp.ensemblgenomes.org)... 193.62.193.161
Connecting to ftp.ensemblgenomes.org (ftp.ensemblgenomes.org)|193.62.193.161|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/bacteria/release-45/fasta/bacteria_0_collection/escherichia_coli_str_k_12_substr_mg1655/dna ... done.
==> SIZE Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa.gz ... 1442049
==> PASV ... done.    ==> RETR Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa.gz ... done.
Length: 1442049 (1.4M) (unauthoritative)

Escherichia_coli_st 100%[===================>]   1.38M  1.34MB/s   

### 1.3 Align Reads to Reference
Now, let's align the reads to the reference genome using BWA:


In [15]:
!sudo apt-get install bwa

# Index the reference genome
!bwa index Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa

# Align reads
!bwa mem Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa SRR1770413_1.fastq SRR1770413_2.fastq > aligned_reads.sam


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bwa is already the newest version (0.7.17-6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
[bwa_index] Pack FASTA... 0.03 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.15 seconds elapse.
[bwa_index] Update BWT... 0.03 sec
[bwa_index] Pack forward-only FASTA... 0.03 sec
[bwa_index] Construct SA from BWT and Occ... 0.70 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa
[main] Real time: 2.044 sec; CPU: 1.941 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 33224 sequences (10000424 bp)...
[M::process] read 33224 sequences (10000424 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (1, 11363, 0, 2)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem

In [ ]:
!sudo apt-get install samtools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2
Suggested packages:
  cwltool
The following NEW packages will be installed:
  libhts3 libhtscodecs2 samtools
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 963 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 samtools amd64 1.13-4 [520 kB]
Fetched 963 kB in 1s (1,457 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: fallin

In [16]:
# Convert SAM to BAM
!samtools view -bS aligned_reads.sam > aligned_reads.bam

# Sort the BAM file
!samtools sort aligned_reads.bam -o aligned_reads_sorted.bam

# Index the BAM file
!samtools index aligned_reads_sorted.bam

Now you have a smaller, manageable BAM file (`aligned_reads_sorted.bam`) to work with for your error detection project.

## 2. Prerequisites
Ensure you have:
- Python 3.7 or later
- Basic understanding of DNA sequencing concepts
- Familiarity with Python programming

## 3. Installation and Setup
Install required libraries:


In [ ]:
!pip install pysam numpy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 92.0 MB/s eta 0:00:00


## 4. Reading BAM Files with Pysam
Let's start by reading our BAM file:


In [17]:
import pysam

def read_bam_file(bam_path):
    try:
        bam_file = pysam.AlignmentFile(bam_path, "rb")
        print(f"Successfully opened BAM file: {bam_path}")
        print(f"Number of mapped reads: {bam_file.mapped}")
        print(f"Number of unmapped reads: {bam_file.unmapped}")
        return bam_file
    except Exception as e:
        print(f"Error opening BAM file: {e}")
        return None

# Usage
bam_path = "aligned_reads_sorted.bam"
bam_file = read_bam_file(bam_path)

if bam_file:
    # Iterate through the first 5 alignments
    for i, read in enumerate(bam_file.fetch()):
        if i >= 5:
            break
        print(f"Read {i+1}: {read.query_name}, Position: {read.reference_start}, CIGAR: {read.cigarstring}")

    bam_file.close()

Successfully opened BAM file: aligned_reads_sorted.bam
Number of mapped reads: 922848
Number of unmapped reads: 365046
Read 1: SRR1770413.4997, Position: 0, CIGAR: 167H134M
Read 2: SRR1770413.21706, Position: 0, CIGAR: 30S271M
Read 3: SRR1770413.36635, Position: 0, CIGAR: 183H118M
Read 4: SRR1770413.38633, Position: 0, CIGAR: 37S264M
Read 5: SRR1770413.50710, Position: 0, CIGAR: 301M


## 5. Error Detection Algorithms
Let's implement some error detection algorithms using our E. coli dataset:




In [18]:
import pysam
import numpy as np

def detect_mismatches(bam_file, reference_genome, chromosome, start, end):
    mismatches = []
    for pileup_column in bam_file.pileup(chromosome, start, end):
        ref_base = reference_genome.fetch(chromosome, pileup_column.reference_pos, pileup_column.reference_pos + 1).upper()
        for pileup_read in pileup_column.pileups:
            if not pileup_read.is_del and not pileup_read.is_refskip:
                read_base = pileup_read.alignment.query_sequence[pileup_read.query_position].upper()
                if read_base != ref_base:
                    mismatches.append((pileup_column.reference_pos, ref_base, read_base))
    return mismatches

def detect_indels(bam_file, chromosome, start, end):
    indels = []
    for pileup_column in bam_file.pileup(chromosome, start, end):
        for pileup_read in pileup_column.pileups:
            if pileup_read.indel != 0:
                indels.append((pileup_column.reference_pos, pileup_read.indel))
    return indels

In [19]:
bam_path = "aligned_reads_sorted.bam"
reference_path = "Escherichia_coli_str_k_12_substr_mg1655.ASM584v2.dna.toplevel.fa"

bam_file = pysam.AlignmentFile(bam_path, "rb")
reference_genome = pysam.FastaFile(reference_path)

# For E. coli, we'll use the entire genome as it's much smaller
chromosome = "Chromosome"  # This may vary depending on how the E. coli genome is labeled in your reference
start = 0
end = reference_genome.get_reference_length(chromosome)

mismatches = detect_mismatches(bam_file, reference_genome, chromosome, start, end)
indels = detect_indels(bam_file, chromosome, start, end)

print(f"Found {len(mismatches)} mismatches and {len(indels)} indels.")

bam_file.close()
reference_genome.close()

Found 195039 mismatches and 4523 indels.


## 6. Analyzing Error Patterns
Now, let's analyze the error patterns we've detected:


In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_mismatch_types(mismatches):
    mismatch_types = {}
    for _, ref_base, read_base in mismatches:
        mismatch_type = f"{ref_base}>{read_base}"
        mismatch_types[mismatch_type] = mismatch_types.get(mismatch_type, 0) + 1
    return mismatch_types

def analyze_indel_sizes(indels):
    indel_sizes = [indel[1] for indel in indels]
    return indel_sizes

# Analyze mismatch types
mismatch_types = analyze_mismatch_types(mismatches)

# Plot mismatch types
plt.figure(figsize=(12, 6))
sns.barplot(x=list(mismatch_types.keys()), y=list(mismatch_types.values()))
plt.title("Mismatch Types in E. coli Genome")
plt.xlabel("Mismatch Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("ecoli_mismatch_types.png")
plt.close()

# Analyze indel sizes
indel_sizes = analyze_indel_sizes(indels)

# Plot indel size distribution
plt.figure(figsize=(12, 6))
sns.histplot(indel_sizes, bins=20, kde=True)
plt.title("Indel Size Distribution in E. coli Genome")
plt.xlabel("Indel Size")
plt.ylabel("Count")
plt.savefig("ecoli_indel_sizes.png")
plt.close()

print("Analysis complete. Check the generated PNG files for visualizations.")

Analysis complete. Check the generated PNG files for visualizations.


## 7. Implementing Quality Control Measures
Let's implement some quality control measures to filter out low-quality reads:


In [21]:
def filter_low_quality_reads(bam_file, min_mapping_quality=20, min_base_quality=20):
    filtered_reads = 0
    total_reads = 0

    for read in bam_file.fetch():
        total_reads += 1
        if read.mapping_quality >= min_mapping_quality and \
           np.mean([ord(q) - 33 for q in read.qual]) >= min_base_quality:
            yield read
        else:
            filtered_reads += 1

    print(f"Filtered {filtered_reads} out of {total_reads} reads.")

# Usage
bam_file = pysam.AlignmentFile(bam_path, "rb")
filtered_bam_path = "filtered_aligned_reads.bam"

with pysam.AlignmentFile(filtered_bam_path, "wb", template=bam_file) as filtered_bam:
    for read in filter_low_quality_reads(bam_file):
        filtered_bam.write(read)

bam_file.close()

# Index the filtered BAM file
pysam.index(filtered_bam_path)

print(f"Created filtered BAM file: {filtered_bam_path}")

Filtered 56287 out of 926112 reads.
Created filtered BAM file: filtered_aligned_reads.bam


## 8. Comparative Analysis
Now, let's compare the error rates before and after quality control:


In [22]:
def calculate_error_rate(bam_path, reference_path, chromosome, start, end):
    bam_file = pysam.AlignmentFile(bam_path, "rb")
    reference_genome = pysam.FastaFile(reference_path)

    mismatches = detect_mismatches(bam_file, reference_genome, chromosome, start, end)
    indels = detect_indels(bam_file, chromosome, start, end)

    total_bases = sum(read.query_length for read in bam_file.fetch(chromosome, start, end))
    error_rate = (len(mismatches) + len(indels)) / total_bases

    bam_file.close()
    reference_genome.close()

    return error_rate

# Calculate error rates
original_error_rate = calculate_error_rate(bam_path, reference_path, chromosome, start, end)
filtered_error_rate = calculate_error_rate(filtered_bam_path, reference_path, chromosome, start, end)

print(f"Original error rate: {original_error_rate:.6f}")
print(f"Filtered error rate: {filtered_error_rate:.6f}")

# Visualize the comparison
labels = ['Original', 'Filtered']
error_rates = [original_error_rate, filtered_error_rate]

plt.figure(figsize=(8, 6))
sns.barplot(x=labels, y=error_rates)
plt.title("Error Rate Comparison in E. coli Genome")
plt.ylabel("Error Rate")
plt.savefig("ecoli_error_rate_comparison.png")
plt.close()

print("Comparison complete. Check the generated PNG file for visualization.")

Original error rate: 0.000717
Filtered error rate: 0.000626
Comparison complete. Check the generated PNG file for visualization.
